#***Carlos Gross-Martinez***
#***Reinforcement Learning***
#***Link to Google Collab Notebook***
#https://colab.research.google.com/drive/14hB9zD2T_AmFcLSHYLDm0HG4WVFojXKJ?usp=sharing

In [2]:
#import library
import numpy as np
import pandas as pd

#function that translate corrdinates to states
def states (coordinates):

    if coordinates[0] == 0 and coordinates[1] == 0: #top left cell
        state = "s0"

    elif coordinates[0] == 0 and coordinates[1] == 1: #top center cell / final state R = -80
        state = "s1"

    elif coordinates[0] == 0 and coordinates[1] == 2: #top right cell / final state R = +100
        state = "s2"

    elif coordinates[0] == 1 and coordinates[1] == 0: #middle left cell
        state = "s3"

    elif coordinates[0] == 1 and coordinates[1] == 1: #middle center cell
        state = "s4"
 
    elif coordinates[0] == 1 and coordinates[1] == 2: #middle right cell
        state = "s5"

    elif coordinates[0] == 2 and coordinates[1] == 0: #bottom left cell / final state R = 25
        state = "s6"

    elif coordinates[0] == 2 and coordinates[1] == 1: #bottom center cell / final state R = -100
        state = "s7"

    else:
        state = "s8" #bottom right cell / final state R = +80

    return state

#function that calculates the possibkle actions based on states
def possible_actions(current_state):

    #actions: up 1, down = 2, left = 3, right = 4  
    actions = (1, 2, 3, 4)
    
    return actions

#function that calculates the next state
def calc_next_state(act, cur_loc):

    temp = np.zeros(2)

    #condition that provides coordinates of next state
    #actions: up 1, down = 2, left = 3, right = 4
    if act == 1:
        if cur_loc[0] > 0:
            temp[1] = cur_loc[1]
            temp[0] = cur_loc[0] - 1
            
    elif act == 2:
        if cur_loc[0] < 2:
            temp[1] = cur_loc[1]
            temp[0] = cur_loc[0] + 1

    elif act == 3:
        if cur_loc[1] > 0:
            temp[0] = cur_loc[0]
            temp[1] = cur_loc[1] - 1
    else:
        if cur_loc[1] < 2:
            temp[0] = cur_loc[0]
            temp[1] = cur_loc[1] + 1

    return temp


#function that returns the index of the state in qSA table     
def cal_state_index (curr_st, qSA):

    for i in range(len(qSA)):
        if qSA[i][0] == curr_st:
            return i

#function that compares all rewards from actions in next state and returns max value
def get_max_act_next_st(next_state, qSA):

    temp_max = 0

    for i in range (len(qSA)):
        if qSA[i][0] == next_state:
            for j in range(1, 5):                
                if temp_max < qSA[i][j]:
                    return qSA[i][j]

    return temp_max

#function that updates the qSA table and calculates error
def update_qSA(curr_st, next_state, action, qSA, rewards):

    alpha = 0.5
    gamma = 0.5

    cur_st_id_qSA = cal_state_index(curr_st, qSA)

    #Qlearning equation calculation
    qSA[cur_st_id_qSA][action] = ((1 - alpha) * qSA[cur_st_id_qSA][action]) + alpha * (rewards[cur_st_id_qSA][action] + (gamma * get_max_act_next_st(next_state, qSA)))

    #calculating error of current iteration
    current_iter_error.append(abs(get_max_act_next_st(next_state, qSA) - qSA[cur_st_id_qSA][action]))

#funnction that randomly selects an 30% exploration and 70% greedy
def select_action(cur_cdnt, curr_st, poss_act, qSA, rewards):

    #condition that selects greedy or exploration actions
    if np.random.uniform(0, 1) <= 0.3:

        #if the condition is exploration, chose random action
        rand_action = np.random.choice(poss_act)
        next_state_coordinate = calc_next_state(rand_action, cur_cdnt)
        next_state = states(next_state_coordinate)
        update_qSA(curr_st, next_state, rand_action, qSA, rewards)
        return rand_action

    else:

        #if the condition is greedy, traverse through all possible actions
        #to determine best action
        for actions in poss_act:

            next_state_coordinate = calc_next_state(actions, cur_cdnt)
            next_state = states(next_state_coordinate)
            update_qSA(curr_st, next_state, actions, qSA, rewards)

    #selecting a ramdon action initially
    action = np.random.choice(poss_act)
    cur_st_id = cal_state_index (curr_st, qSA)

    #setting temp_max to initial qSA value
    temp_max = qSA[cur_st_id][1]

    #traversing through qSA value for action to find the greediest
    for i in range(2, 5):

        if temp_max < qSA[cur_st_id][i]:
            action = i
            temp_max = qSA[cur_st_id][i]

    return action

#function that checks if the current state is a goal stater
def is_goal (current_state, goal_state):

    return all(current_state == goal_state)

#function that print qSA table based on iteration
def printqSA(iter_count):

    df = pd.DataFrame(qSA, columns = ['State', 'Up', 'Down', 'Left', 'Right'])

    print('')
    print('Q(S,A) Table on iteration ' + str(iter_count) + ':')
    print(df)

##########################################################
#Code starts
##########################################################

#actions: up 1, down = 2, left = 3, right = 4
#declaration and initialization of rewards table
rewards = (("s0", 0, 0, 0, -80),
           ("s1", 0, 0, 0, 0),
           ("s2", 0, 0, 0, 0),
           ("s3", 0, 25, 0, 0),
           ("s4", -80, -100, 0, 0),
           ("s5", 100, 80, 0, 0),
           ("s6", 0, 0, 0, 0),
           ("s7", 0, 0, 0, 0),
           ("s8", 0, 0, 0, 0),
          )

#declaration and initialization of qSA table
qSA = [["s0", 0, 0, 0, 0],
       ["s1", 0, 0, 0, 0],
       ["s2", 0, 0, 0, 0],
       ["s3", 0, 0, 0, 0],
       ["s4", 0, 0, 0, 0],
       ["s5", 0, 0, 0, 0],
       ["s6", 0, 0, 0, 0],
       ["s7", 0, 0, 0, 0],
       ["s8", 0, 0, 0, 0],
      ]

#declaration and initialization of variables
goal_coordinate = [[0,1], [0,2], [2,0], [2,1], [2,2]]
start_coordinate = [0,0]
current_state_coordinate = []

iteration = []
error_list = []
steps_list = []

#conducting 1000 iterations of program
for i in range(0, 1000):

    #declaration and initialization of variables
    steps = 0
    found_goal = False
    current_iter_error = []
    current_state_coordinate = start_coordinate

    #loop that will not stop until goal coordinate is entered
    while found_goal!= True:

        #determining current state from coordinates
        curr_st = states(current_state_coordinate)

        #calculating possible moves based on location
        poss_act = possible_actions(curr_st)

        #determinig action to complete
        action = select_action(current_state_coordinate, curr_st, poss_act, qSA, rewards)

        #updating coordinates to the ones in thext state
        current_state_coordinate = calc_next_state(action, current_state_coordinate)

        #counting the steps taken by algorithm
        steps += 1

        #checking if the step is the goal state after making the move
        for goals in goal_coordinate:

            temp_goal_state = np.array(goals)

            #setting found_goal value based on boolean operation results
            found_goal = is_goal(current_state_coordinate, temp_goal_state)

            #exit loop if arrived at goal state
            if found_goal == True:
                break

    #appending information to list to graph
    iteration.append(i)
    steps_list.append(steps)
    error_list.append(max(current_iter_error))  

    #print current qSA table based on the current iteration
    if i ==  1 or i == 5 or i == 10:
        printqSA(i)

#print final qSA table
df = pd.DataFrame(qSA, columns = ['State', 'Up', 'Down', 'Left', 'Right'])
print('')
print('Q(S,A) Final Table:')
print(df)


Q(S,A) Table on iteration 1:
  State   Up  Down  Left  Right
0    s0  0.0   0.0   0.0  -75.0
1    s1  0.0   0.0   0.0    0.0
2    s2  0.0   0.0   0.0    0.0
3    s3  0.0  12.5   0.0    0.0
4    s4  0.0   0.0   0.0    0.0
5    s5  0.0   0.0   0.0    0.0
6    s6  0.0   0.0   0.0    0.0
7    s7  0.0   0.0   0.0    0.0
8    s8  0.0   0.0   0.0    0.0

Q(S,A) Table on iteration 5:
  State        Up       Down      Left  Right
0    s0  0.329590   0.674438  0.137329 -78.75
1    s1  0.000000   0.000000  0.000000   0.00
2    s2  0.000000   0.000000  0.000000   0.00
3    s3  0.344849  24.218750  0.344849   0.00
4    s4  0.000000   0.000000  0.000000   0.00
5    s5  0.000000   0.000000  0.000000   0.00
6    s6  0.000000   0.000000  0.000000   0.00
7    s7  0.000000   0.000000  0.000000   0.00
8    s8  0.000000   0.000000  0.000000   0.00

Q(S,A) Table on iteration 10:
  State        Up       Down      Left      Right
0    s0  0.104284   0.134814  0.081861 -79.921875
1    s1  0.000000   0.000000 